In [1]:
import Random#, logging
using  Plots, CSV, Gen#, Dates
using DataFrames: DataFrame
# Disable logging, because @animate is verbose otherwise
#Logging.disable_logging(Logging.Info);

In [ ]:
SubChunkSize = 20
@gen function regression_with_outliers(xs::Vector{<:Real})
    # First, generate some parameters of the model. We make these
    # random choices, because later, we will want to infer them
    # from data. The distributions we use here express our assumptions
    # about the parameters: we think the slope and intercept won't be
    # too far from 0; that the noise is relatively small; and that
    # the proportion of the dataset that don't fit a linear relationship
    # (outliers) could be anything between 0 and 1.
    n = length(xs)
    NumChunks = div(n, SubChunkSize, RoundUp)
    Indx = 1
    Buffer_x = 0
    Buffer_y = 0
    mu = 0
    prob_outlier = {:prob_outlier} ~ uniform(0, .2)
    for i = 1:NumChunks
        Quad = {(:Quad, i)} ~ normal(0, 300)
        slope = {(:slope, i)} ~ normal(0, 3000)
        noise = {(:noise, i)} ~ gamma(200, 200)
        # Next, we generate the actual y coordinates.
         ys = Float64[]
        for j = 1:SubChunkSize
            # Decide whether this point is an outlier, and set
            # mean and standard deviation accordingly
            if(!(Indx > n))
                if ({:data => Indx => :is_outlier} ~ bernoulli(prob_outlier))
                    (mu, std) = (1000*(xs[Indx] - Buffer_x) * slope + Buffer_y,  100*noise)
                else
                    (mu, std) = ((xs[Indx] - Buffer_x) * slope + Buffer_y,  noise)
                end
                # Sample a y value for this point
                push!(ys, {:data => Indx => :y} ~ normal(mu, std))
                Indx += 1
            end
        end
        Buffer_y = mu
        Buffer_x = xs[Indx-1]
    end 
    ys
end;

In [ ]:
using Plots
#length(floor(xs/40)+1)
function serialize_trace(trace)
    (xs,) = Gen.get_args(trace)
    n = length(xs)
    NumChunks = div(n, SubChunkSize, RoundUp)
    #print(xs)
    Dict(:slope => [trace[(:slope, i)] for i in 1:NumChunks],
         :inlier_std => [trace[(:noise, i)] for i in 1:NumChunks],
         :points => zip(xs, [trace[:data => i => :y] for i in 1:length(xs)]),
         :xs => xs,
         :outliers => [trace[:data => i => :is_outlier] for i in 1:length(xs)])
end

In [ ]:
function visualize_trace(trace::Trace; title="")
    trace = serialize_trace(trace)
    n = length(trace[:xs])
    NumChunks = div(n, SubChunkSize, RoundUp)
    outliers = [pt for (pt, outlier) in zip(trace[:points], trace[:outliers]) if outlier]
    inliers =  [pt for (pt, outlier) in zip(trace[:points], trace[:outliers]) if !outlier]
    PLT = Plots.scatter(map(first, inliers), map(last, inliers), markercolor="blue", label=nothing, title=title) 
    PLT = Plots.scatter!(map(first, outliers), map(last, outliers), markercolor="red", label=nothing)
    Buffer_y = 0
    Buffer_x = 0
    for i = 1:NumChunks
        inferred_line(x) = trace[:slope][i] * (x - Buffer_x) + Buffer_y
        if(1 + (i-1)*SubChunkSize > n)
            left_x = trace[:xs][n]
        else
            left_x = trace[:xs][1 + (i-1)*SubChunkSize]
        end
        left_y  = inferred_line(left_x)
        if(SubChunkSize*i > n)
            right_x = trace[:xs][n]
        else
            right_x = trace[:xs][SubChunkSize*i]
        end
        right_y = inferred_line(right_x)
        PLT = Plots.plot!([left_x, right_x], [left_y, right_y], color = "black", lw = 3, label = nothing)
        Buffer_y = right_y
        Buffer_x = right_x
    end
    return PLT
end
#, xlims=[MinX, MaxX], ylims=[MinY, MaxY]

In [ ]:
dataframe = CSV.read("../Data/Proccesed/DetrendedCov.csv", DataFrame)

In [ ]:
#using DataArrays
dataframe[!,:N1] = convert.(Base.Float64,dataframe[!,:N1])
dataframe[!,:Date] = convert.(Base.Float64,dataframe[!,:Date])

xs = dataframe."Date"[1:390]
ys = dataframe."N1"[1:390]

Plots.scatter(xs, ys, color="black", xlabel="X", ylabel="Y", 
              label=nothing, title="Observations - regular data and outliers")

In [ ]:
function make_constraints(ys::Vector{Float64})
    constraints = Gen.choicemap()
    for i=1:length(ys)
        constraints[:data => i => :y] = ys[i]
    end
    constraints
end;

In [ ]:
observations = make_constraints(ys);

In [ ]:
function logmeanexp(scores)
    logsumexp(scores) - log(length(scores))
end;

In [ ]:
traces    = [first(Gen.importance_resampling(regression_with_outliers, (xs,), observations, 1000)) for i in 1:9]
#log_probs = [get_score(t) for t in traces]
#println("Average log probability: $(logmeanexp(log_probs))")
Plots.plot([visualize_trace(t) for t in traces]...)

In [ ]:
# Perform a single block resimulation update of a trace.
function block_resimulation_update(tr)
    (xs,) = get_args(tr)
    n = length(xs)
    NumChunks = div(n, SubChunkSize, RoundUp)
    for j=1:NumChunks
        # Block 1: Update the line's parameters
        line_params = select((:noise,j), (:slope,j))
        (tr, _) = mh(tr, line_params)
    end
    
    # Blocks 2-N+1: Update the outlier classifications
    for i=1:n
        (tr, _) = mh(tr, select(:data => i => :is_outlier))
    end
    
    # Block N+2: Update the prob_outlier parameter
    (tr, _) = mh(tr, select(:prob_outlier))
    
    # Return the updated trace
    tr
end;

In [ ]:
function block_resimulation_inference(xs, ys)
    observations = make_constraints(ys)
    (tr, W) = generate(regression_with_outliers, (xs,), observations)
    for iter=1:100
        tr = block_resimulation_update(tr)
    end
    tr
end;

In [ ]:
scores = Vector{Float64}(undef, 3)
for i=1:3
    @time tr = block_resimulation_inference(xs, ys)
    scores[i] = get_score(tr)
end;
println("Log probability: ", logmeanexp(scores))

In [ ]:
scores = Vector{Float64}(undef, 3)
for i=1:3
    @time (tr, W) = importance_resampling(regression_with_outliers, (xs,), observations, 170)
    scores[i] = get_score(tr)
end;
println("Log probability: ", logmeanexp(scores))

In [ ]:
t, = generate(regression_with_outliers, (xs,), observations)

viz = Plots.@animate for i in 1:300
    global t
    t = block_resimulation_update(t)
    visualize_trace(t; title="Iteration $i/300")
end;
gif(viz)

In [ ]:
t, = generate(regression_with_outliers, (xs,), observations)

viz = Plots.@animate for i in 1:200
    global t
    t = block_resimulation_update(t)
    visualize_trace(t; title="Iteration $i/200")
end;
gif(viz)